In [ ]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.stats import zscore

file = '../data/01_raw/top_500.csv'

try:
     data = pd.read_csv(file)
     print(f"Plik '{file}' został poprawnie wczytany.")
except Exception as e:
     print(f"Błąd podczas wczytywania pliku: {e}")
     data = pd.DataFrame()

In [ ]:
# Podstawowe informacje o danych
info = pd.DataFrame({
    'Typ kolumny': data.dtypes,
    'Brakujące wartości': data.isnull().sum(),
    'Unikalne wartości': data.nunique()
})

print(f"Rozmiar danych: {data.shape[0]} wierszy × {data.shape[1]} kolumn\n")
display(info)